In [1]:
import numpy as np
import polars as pl
import sys

sys.path.append("../")

from multiprocessing import cpu_count

from src.img_db import ImageDatabase

In [3]:
# img_db = ImageDatabase("../data/images/original", "../data/images/generated")
# img_db.load_files()

Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: bad Huffman code


In [4]:
# img_db.denoise()

In [5]:
# img_db.crop(crop_size=64)

# X_raw, y = img_db.export(log_abs=False)

In [2]:
import joblib

In [227]:
joblib.dump(img_db.get_data()[0], "../data/all_artworks_denoised_crop64_list.lzma", compress=8, protocol=5)

['../data/all_artworks_denoised_crop64_list.lzma']

In [7]:
# pl.from_numpy(
#     X_raw, 
#     schema=[f"c{i}" for i in range(X_raw.shape[1])], 
#     orient="row"
# ).write_parquet("../data/all_artworks_denoised_zstd.parquet", compression="zstd", compression_level=19)

In [2]:
X_raw2 = pl.read_parquet("../data/intermediate_data/all_artworks_denoised.parquet").to_numpy()

img_db2 = ImageDatabase("../data/images/original", "../data/images/generated")
img_db2.load_files()
_, y = img_db2.get_data()

FileNotFoundError: No such file or directory: ../data/intermediate_data/all_artworks_denoised.parquet

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve, balanced_accuracy_score, f1_score, make_scorer
from sklearn.preprocessing import StandardScaler

In [5]:
X_raw2.shape

(13963, 4096)

In [11]:
y = y.astype(np.int_)
# X = X_raw2
X = np.log(np.abs(X_raw2 + 1))[:, np.r_[:400, -1000:-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234, stratify=y)

In [7]:
# X_train = X_train[:, np.r_[:512, -512:-0]]
# X_test = X_test[:, np.r_[:512, -512:-0]]
# X_train = X_train[:, :1024]
# X_test = X_test[:, :1024]

X_test.shape

(2793, 1299)

In [194]:
# m1 = LogisticRegression(
#     class_weight="balanced",
#     random_state=1234,
#     n_jobs=cpu_count() - 2,
#     solver="saga",
#     max_iter=800
# ).fit(X_train, y_train)
m1 = LogisticRegressionCV(
    penalty="l2",
    cv=StratifiedKFold(n_splits=5, random_state=1234, shuffle=True),
    class_weight="balanced",
    random_state=1234,
    n_jobs=cpu_count() - 2,
    solver="saga",
    max_iter=500
).fit(X_train, y_train)

/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10

In [195]:
pred_proba1 = m1.predict_proba(X_test)[:, 1]
pred1 = (pred_proba1 > 0.5).astype(np.int_)

In [196]:
balanced_accuracy_score(y_test, pred1)

0.6627637867190561

In [197]:
roc_auc_score(y_test, pred_proba1)

0.7241097198231856

In [6]:
import optuna

In [12]:
m2 = SVC(
    class_weight="balanced"
).fit(X_train, y_train)

In [14]:
import joblib

joblib.dump(m2, "../output/svm_trained.xz", compress=9, protocol=5)

['../output/svm_trained.xz']

In [13]:
# m2 = SVC(
#     class_weight="balanced",
#     **{'kernel': 'poly', 'C': 1.5546279362233293, 'gamma': 'scale'}
# ).fit(X_train, y_train)

pred2 = m2.predict(X_test)

print(balanced_accuracy_score(y_test, pred2))
print(f1_score(y_test, pred2))

0.7256004183061361
0.8329177057356608


In [13]:
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=1234)

def obj(trial: optuna.Trial):
    mod = SVC(
        kernel = trial.suggest_categorical("kernel", ["poly", "rbf", "sigmoid"]),
        C = trial.suggest_float('C', 0.1, 3, log=True),
        gamma=trial.suggest_categorical('gamma', ['auto','scale']),
        class_weight="balanced",
        max_iter=1500
    )

    return cross_val_score(mod, X_train, y_train, cv=kfold, n_jobs=cpu_count() - 2, scoring=make_scorer(balanced_accuracy_score)).mean()

In [14]:
study = optuna.create_study(direction="maximize", storage="sqlite:///svm_tuning_trials.db")
study.optimize(obj, n_trials=40, show_progress_bar=True)
print(study.best_trial)

[I 2023-02-23 08:23:50,279] A new study created in RDB with name: no-name-eaae91a0-216f-49d7-be8a-2bc482dc5196
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/40 [00:00<?, ?it/s]

/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Cons

[I 2023-02-23 08:24:47,973] Trial 0 finished with value: 0.5 and parameters: {'kernel': 'rbf', 'C': 0.5756086951124922, 'gamma': 'auto'}. Best is trial 0 with value: 0.5.


/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Cons

...

[I 2023-02-23 08:49:15,251] Trial 37 finished with value: 0.5693810136770382 and parameters: {'kernel': 'poly', 'C': 2.5135804066575003, 'gamma': 'auto'}. Best is trial 21 with value: 0.5858078713922666.


/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Cons

[I 2023-02-23 08:50:09,164] Trial 38 finished with value: 0.5 and parameters: {'kernel': 'rbf', 'C': 0.6778768500268589, 'gamma': 'auto'}. Best is trial 21 with value: 0.5858078713922666.


/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1500).  Cons

[I 2023-02-23 08:50:48,599] Trial 39 finished with value: 0.5445050741460967 and parameters: {'kernel': 'poly', 'C': 1.5546279362233293, 'gamma': 'scale'}. Best is trial 21 with value: 0.5858078713922666.
FrozenTrial(number=21, state=TrialState.COMPLETE, values=[0.5858078713922666], datetime_start=datetime.datetime(2023, 2, 23, 8, 37, 41, 332911), datetime_complete=datetime.datetime(2023, 2, 23, 8, 38, 19, 910161), params={'C': 0.38247176315704867, 'gamma': 'auto', 'kernel': 'poly'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'C': FloatDistribution(high=3.0, log=True, low=0.1, step=None), 'gamma': CategoricalDistribution(choices=('auto', 'scale')), 'kernel': CategoricalDistribution(choices=('poly', 'rbf', 'sigmoid'))}, trial_id=26, value=None)
